In [89]:
import pandas as pd
df = pd.read_csv('data/dog_adopt_eda.csv')

In [90]:
df.head()

,품종,색,성별,중성화유무,무게(Kg),기타특징,상태,나이
0,비숑 프리제,흰색,수컷,아니오,5.1,칩 있음,반환,152
1,보더 콜리,흰+검,암컷,예,20.0,"칩 등록되어 있음, 사람을 좋아함",반환,56
2,포메라니안,갈색,수컷,아니오,5.7,"칩있음, 하네스 착용",반환,116
3,보더 콜리,치즈,미상,미상,15.0,순함,반환,68
4,믹스견,검정,수컷,아니오,9.3,"순함, 겁 많음",미입양,44


In [91]:
df_encoded = pd.get_dummies(df, columns=['성별', '중성화유무', '상태'])

In [92]:
df = df_encoded

In [93]:
from sentence_transformers import SentenceTransformer
import torch 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
embeddings = model.encode(df['색'].tolist(), device=device, convert_to_tensor=True)
    

c:\Users\Admin\Desktop\metaverse\project\dog_adopt_probability\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [98]:
embeddings_np = embeddings.cpu().numpy()

RuntimeError: Numpy is not available

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(embeddings_np)
print(clusters)

NameError: name 'embeddings_np' is not defined

In [32]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-3]
y = df.iloc[:,-3:]
print(X.shape)  # 피처 데이터의 크기 출력
print(y.shape)  # 타겟 데이터의 크기 출력
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(2496, 11)
(2496, 3)


In [41]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train[['나이', '무게(Kg)']] = scaler.fit_transform(X_train[['나이', '무게(Kg)']])
X_test[['나이', '무게(Kg)']] = scaler.transform(X_test[['나이', '무게(Kg)']])


In [99]:
df['기타특징']

0                            칩 있음
1              칩 등록되어 있음, 사람을 좋아함
2                     칩있음, 하네스 착용
3                              순함
4                        순함, 겁 많음
                  ...            
2491     동물등록돼있음. 빨간목줄착용, 결막염 있음.
2492             소방서 인계 후 구조. 순함.
2493          생후 6개월 강아지. 백내장 의심.
2494    동물등록돼있음. 소방서 인계 후 구조. 순함.
2495             진도믹스추정, 경계심이 많음.
Name: 기타특징, Length: 2496, dtype: object

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X_train['품종'] = label_encoder.fit_transform(X_train['품종'])
X_test['품종'] = label_encoder.fit_transform(X_test['품종'])

,품종,색,무게(Kg),기타특징,나이,성별_미상,성별_수컷,성별_암컷,중성화유무_미상,중성화유무_아니오,중성화유무_예
109,13,미색,0.009350,"15마리 함께 구조, 3주 추정",0.020,False,True,False,False,True,False
2301,13,검은색,0.065004,"관리.1551 의 자견, 생후 70일 강아지",0.045,False,False,True,False,True,False
354,22,흰색,0.109528,"2마리 함께 구조, 털 뭉침 심함",0.120,False,False,True,True,False,False
266,13,흰색,0.332146,"부견,모견1두 새끼 3두 함께 구조, 8개월 추정",0.005,False,True,False,False,True,False
2099,13,검황색,0.443455,9 아이 함께 구조. 순함.,0.180,False,False,True,True,False,False


In [102]:
dog_color = pd.read_csv('data/dog_color.csv')
df.drop(columns=['상태_미입양', '상태_반환', '상태_입양'], inplace=True)

In [104]:
df_condition = pd.read_csv('data/dog_adopt_eda.csv')['상태']
df_condition
df = pd.concat([df,df_condition], axis=1)

In [109]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['색'] = label_encoder.fit_transform(df['색'])

In [107]:
df['색'] = dog_color

In [108]:
df['보유']

,품종,색,무게(Kg),기타특징,나이,성별_미상,성별_수컷,성별_암컷,중성화유무_미상,중성화유무_아니오,중성화유무_예,상태
0,비숑 프리제,흰색계열,5.1,칩 있음,152,False,True,False,False,True,False,1
1,보더 콜리,혼합색계열,20.0,"칩 등록되어 있음, 사람을 좋아함",56,False,False,True,False,False,True,1
2,포메라니안,갈색계열,5.7,"칩있음, 하네스 착용",116,False,True,False,False,True,False,1
3,보더 콜리,흰색계열,15.0,순함,68,True,False,False,True,False,False,1
4,믹스견,검정색계열,9.3,"순함, 겁 많음",44,False,True,False,False,True,False,0
